# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [129]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))

save (generic function with 1 method)

In [132]:
function basic_capex(buses, lines, gens, loads, variability, P)
    """
    Function to solve DC OPF problem 
    Inputs:
        gens -- dataframe with generator info and costs
        lines -- dataframe with transmission lines info
        buses -- dataframe with bus types and loads
    """   
    #### HELPER FUNCTIONS ####
    # Select a line column using i and j 
    line = (i, j, col) -> lines[lines_map[(i, j)], col]
    # Subset of all lines connected to bus i
    J = i -> lines.t_bus[lines.f_bus .== i]
    # Select all generators in G connected to bus i
    sel = (G, i) -> G[gens[G,:bus] .== i]
    # Return the last time index with circular wrapping
    function prev(t)
        P_i = findfirst(P_i -> t in P_i, P)
        shift = first(P[P_i])
        p = length(P[P_i])
        return mod1(t - shift, p) + shift - 1
    end
    
    ##### SETS ####
    G = 1:nrow(gens)             # G: Set of all generators
    G_s = G[gens.ess .== 1]      # G_s: Set of all energy storage system (ESS) generators
    G_c = G[gens.canidate .== 1] # G_c: Set of generators that can be expanded
    
    N = 1:nrow(buses)            # N: Set of all network nodes
    L = collect(zip(             # L: Set of all lines/branches
        lines.f_bus, lines.t_bus))

    
    T = 1:ncol(loads)            # T: Set of all hours
                                 # P: Set of representative periods
    W = length(P)

    #### GLOBALS ####
    # Mapping from (i, j) tuple to line index l
    lines_map = Dict(zip(L, 1:length(L)))
    
    #### PARAMETERS ####
    θlim = π*(60/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    
    storage_hrs = 4    # Storage Hours * Power Capacity = Energy capacity ∀ g ∈ G_s
    η_charge = 0.97    # Charging efficiency  
    η_discharge = 0.91 # Discharging efficiecy

    # Toy parameters for a limited investment in solar at each bus
    shed_cost = 1000   # Cost of load sheding
    budget = 0
    solar_cost = 0.1475
    
    ########################
    #####     MODEL    #####
    ########################
    
    Expansion_Model = Model(() -> Gurobi.Optimizer())
    #set_optimizer_attribute(Expansion_Model, "NumericFocus", 2)
    
    # Decision variables   
    @variables(Expansion_Model, begin
        GEN[G,T]           # Generation of each generator 
        SHED[N,T] ≥ 0      # Load sheading at bus N
        THETA[N,T]         # Voltage phase angle of bus
        FLOW[L,T]          # Flows between all pairs of nodes
        SOC[G_s,T] ≥ 0     # ESS state of charge
        CHARGE[G_s,T] ≥ 0  # Chargeing of ESS 
    end)
    
    # Objective function (Note: Using just the linear part of quadratic cost)
    @objective(Expansion_Model, Min,
            sum(gens[g,:c0] + gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T) 
            + sum(shed_cost * SHED[i,t] for i ∈ N, t ∈ T) #TODO expressions
    ) 
    
    # Supply demand balances: sum(generation) + shedding - demand - sum(charging) = sum(flows)
    @constraint(Expansion_Model, cBalance[i ∈ N, t ∈ T], 
        sum(GEN[g,t] for g ∈ sel(G, i))
        + SHED[i,t] - loads[i,t] 
        - sum(CHARGE[g,t] for g ∈ sel(G_s, i))
         == sum(FLOW[(i,j),t] for j ∈ J(i))
    )
    
    # Max and min generation constraints
    @constraint(Expansion_Model, cMaxGen[g ∈ G, t ∈ T], GEN[g,t] ≤ variability[g,t]*gens[g,:pmax]) 
    # TODO, ^ when adding cap slip this constraint based on no expansion and with expansion ... (gens[g,:pmax] +CAP[g]) also add this do SOC constraints
    @constraint(Expansion_Model, cMinGen[g ∈ G, t ∈ T], GEN[g,t] ≥ 0)#gens[g,:pmin])

    # TODO: Add some seprate constraints here for new build solar and ess

    # Load shedding constraint
    @constraint(Expansion_Model, cShed[i ∈ N, t ∈ T], SHED[i,t] ≤ loads[i,t])

    #### BATTERY STORAGE ####
    
    # Energy storage and charging constraints
    @constraint(Expansion_Model, cMaxCharge[g ∈ G_s, t ∈ T], CHARGE[g,t] ≤ gens[g,:pmax])
    @constraint(Expansion_Model, cMaxStateOfCharge[g ∈ G_s, t ∈ T], SOC[g,t] ≤ storage_hrs*gens[g,:pmax])

    # Recursively constrain all SOCs with wrapping for each period
    @constraint(Expansion_Model, cStateOfCharge[g ∈ G_s, t ∈ T],
        SOC[g,t] == SOC[g,prev(t)] + (CHARGE[g,t]*η_charge - GEN[g,t]/η_discharge))
    
    #### DC OPF ####
            
    # Create slack bus with theta=0
    fix.(THETA[slack_bus, T], 0)
    
    # Max line flow constraints
    @constraint(Expansion_Model, cLineLimits[(i,j) ∈ L, t ∈ T], 
        FLOW[(i,j),t] ≤ line(i,j,:rate_a)) 
    
    # Angle limits 
    @constraint(Expansion_Model, cAngleLimitsMax[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≤  θlim)
    @constraint(Expansion_Model, cAngleLimitsMin[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≥ -θlim)
                    
    # Flow constraints on each branch
    @constraint(Expansion_Model, cLineFlows[(i,j) ∈ L, t ∈ T],
        FLOW[(i,j),t] == line(i,j,:sus)*(THETA[i,t] - THETA[j,t]));

    optimize!(Expansion_Model)

    return Expansion_Model
end

basic_capex (generic function with 1 method)

In [134]:
# 3b. Ramping
# 4. Investment decisions


# TODOs
# - Add investment decisions (Just do demand node solar? OR Treat it seprately? Having a hard time managing the complexity...)
# - kmeans clustering
# - Ramping (if time)


## EXTRA
# Quardatic costs



## Questions for Qi ##
# a) costs? How do you handel the fact that costs are quadratic?

In [136]:
run_model(load, basic_capex, save, "san_diego_system", "test_senario")

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1177848 rows, 480312 columns and 2359728 nonzeros
Model fingerprint: 0x78b2a7c4
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 3e+03]
Presolve removed 1034622 rows and 334580 columns
Presolve time: 0.99s
Presolved: 143226 rows, 176459 columns, 633721 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.08s

Barrier statistics:
 Free vars  : 6205
 AA' NZ     : 1.382e+06
 Factor NZ  : 3.769e+06 (roughly 160 MB of memory)
 Factor Ops : 1.259e+08 (less than 1 second per iteratio

"/Users/adamsedlak/Documents/Julia/MAE243/final-project/models/san_diego_system/outputs/test_senario/prices.csv"

In [4]:
(buses, lines, gens, loads, variability, P) = load("san_diego_system");

In [100]:
# 30s to solve 2 weeks?
# 2m to solve 5 weeks

Row,bus,fueltype,c2,c1,c0,pmax,ess,canidate,gen_id
,Int64,String,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,5,IMPORT,0.0,31.3952,0.0,200.0,0,0,1
2,7,Conventional Hydroelectric,0.0787244,14.8788,34.6468,1.4,0,0,2
3,12,Natural Gas Fired Combustion Turbine,0.0635901,14.269,300.534,60.5,0,0,3
4,13,Batteries,0.067574,16.3717,401.194,1.5,1,1,4
5,13,Solar Photovoltaic,0.0,0.0,0.0,41.7614,0,1,5
6,15,Onshore Wind Turbine,0.0,0.0,0.0,143.0,0,0,6
7,18,Natural Gas Fired Combustion Turbine,0.0350045,15.1585,428.748,659.0,0,0,7
8,18,Other Waste Biomass,0.0947039,22.4096,554.685,3.2,0,0,8
9,19,Landfill Gas,0.25325,58.907,173.53,5.4,0,0,9


In [8]:
Expansion_Model = Model(() -> Gurobi.Optimizer())
#set_optimizer_attribute(Expansion_Model, "NumericFocus", 2)

# Decision variables   
@variables(Expansion_Model, begin
    GEN[G,T]           # Generation of each generator 
    SHED[N,T] ≥ 0      # Load sheading at bus N
    THETA[N,T]         # Voltage phase angle of bus
    FLOW[L,T]          # Flows between all pairs of nodes
    SOC[G_s,T] ≥ 0     # ESS state of charge
    CHARGE[G_s,T] ≥ 0  # Chargeing of ESS 
end)

# Objective function (Note: Using just the linear part of quadratic cost)
@objective(Expansion_Model, Min,
        sum(gens[g,:c0] + gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T) 
        + sum(shed_cost * SHED[i,t] for i ∈ N, t ∈ T) #TODO expressions
) 

# Supply demand balances: sum(generation) + shedding - demand - sum(charging) = sum(flows)
@constraint(Expansion_Model, cBalance[i ∈ N, t ∈ T], 
    sum(GEN[g,t] for g ∈ sel(G, i))
    + SHED[i,t] - loads[i,t] 
    - sum(CHARGE[g,t] for g ∈ sel(G_s, i))
     == sum(FLOW[(i,j),t] for j ∈ J(i))
)

# Max and min generation constraints
@constraint(Expansion_Model, cMaxGen[g ∈ G, t ∈ T], GEN[g,t] ≤ variability[g,t]*gens[g,:pmax]) 
# TODO, ^ when adding cap slip this constraint based on no expansion and with expansion ... (gens[g,:pmax] +CAP[g]) also add this do SOC constraints
@constraint(Expansion_Model, cMinGen[g ∈ G, t ∈ T], GEN[g,t] ≥ 0)#gens[g,:pmin])

# TODO: Add some seprate constraints here for new build solar and ess

# Load shedding constraint
@constraint(Expansion_Model, cShed[i ∈ N, t ∈ T], SHED[i,t] ≤ loads[i,t])

#### BATTERY STORAGE ####

# Energy storage and charging constraints
@constraint(Expansion_Model, cMaxCharge[g ∈ G_s, t ∈ T], CHARGE[g,t] ≤ gens[g,:pmax])
@constraint(Expansion_Model, cMaxStateOfCharge[g ∈ G_s, t ∈ T], SOC[g,t] ≤ storage_hrs*gens[g,:pmax])



Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22


2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [4, 13, 15, 20, 22, 39, 43, 60, 70, 71  …  352, 353, 354, 355, 356, 357, 358, 359, 360, 361]
    Dimension 2, 1:22
And data, a 152×22 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cMaxStateOfCharge[4,1] : SOC[4,1] ≤ 6       …  cMaxStateOfCharge[4,22] : SOC[4,22] ≤ 6
 cMaxStateOfCharge[13,1] : SOC[13,1] ≤ 4        cMaxStateOfCharge[13,22] : SOC[13,22] ≤ 4
 cMaxStateOfCharge[15,1] : SOC[15,1] ≤ 160      cMaxStateOfCharge[15,22] : SOC[15,22] ≤ 160
 cMaxStateOfCharge[20,1] : SOC[20,1] ≤ 30       cMaxStateOfCharge[20,22] : SOC[20,22] ≤ 30
 cMaxStateOfCharge[22,1] : SOC[22,1] ≤ 6        cMaxStateOfCharge[22,22] : SOC[22,22] ≤ 6
 cMaxStateOfCharge[39,1] : SOC[3

In [60]:
for (i, P_i) in enumerate(P)
    # Number of hours in the representative time period Pᵢ
    p = length(P_i)
    # Recursively constrain all SOCs with wrapping for each period
    
end

LoadError: An object of name cname is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, for example,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :cname)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:cname]`.


In [82]:
@constraint(Expansion_Model, cStateOfCharge[g ∈ G_s, t ∈ T],
        SOC[g,t] == SOC[g,foo(t)] 
        + (CHARGE[g,t]*η_charge - GEN[g,t]/η_discharge),
    );

LoadError: An object of name cStateOfCharge is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, for example,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :cStateOfCharge)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:cStateOfCharge]`.


foo (generic function with 1 method)

In [86]:
for t in T
    println(bax(t))

end

7
1
2
3
4
5
6
22
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [84]:
function bax(t)
    P_i = findfirst(P_i -> t in P_i, P)
    shift = first(P[P_i])
    p = length(P[P_i])
    return mod1(t - shift, p) + shift - 1
end


bax (generic function with 1 method)